In [ ]:
import pandas as pd
#import json
from datetime import date

# Collecting the data from Twitter

In [ ]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])


In [ ]:
import pysad
import pysad.utils
import pysad.collect

In [ ]:
import importlib
importlib.reload(pysad.collect)

In [ ]:
init_accounts = pysad.utils.initial_accounts()

In [ ]:
init_accounts.list()

In [ ]:
######Choose a category##############    
#category_name = 'swiss_climate_controversial'
#category_name = 'swiss_climate_regular'
#category_name = 'french_tech_lesechos'
#category_name = 'swiss_immigration'
#category_name = 'swiss_immigration2'
#category_name = 'debat_burqa'
category_name = 'hackathlon'
category_name = 'hackathlon_popular'
category_name = 'hackathlon_reddit'
category_name = 'hackathlon_missingtweets'

#####################################

username_list = init_accounts.accounts(category_name)

# create the path to save the experiment indexed with the date of today
today = date.today()
date_string = today.strftime("%Y%m%d")
print("date string =", date_string)

tweet_data_path_list = ['tweetdata', category_name, date_string]
results_data_path_list = ['resultsdata2', category_name, date_string]


In [ ]:
tweet_data_path = ''.join(tweet_data_path_list)
results_data_path = ''.join(results_data_path_list)

In [ ]:
# Initialize folders (create or clean them if they exist)
# Set erase=False if you need to keep the previous collection
tweet_data_path = pysad.utils.initialize_folder(tweet_data_path_list, erase=False)
results_data_path = pysad.utils.initialize_folder(results_data_path_list, erase=False)

In [ ]:
# If the list of users is too large:
# Keep a random subset of the list of users
# import random
# username_list_tmp = random.sample(username_list,len(username_list)//10)
# username_list = username_list_tmp

In [ ]:
min_mentions = 2 # minimal number of mentions of a user to be followed
max_day_old = None # number max of days in the past
exploration_depth = 1 # mention of mention of mention of ... up to exploration depth

In [ ]:
import importlib
importlib.reload(pysad.collect)

In [ ]:
total_user_list = pysad.collect.collect_tweets(username_list, tweet_data_path, python_tweets, min_mentions=min_mentions,
               max_day_old=max_day_old, exploration_depth=exploration_depth)

In [ ]:
len(total_user_list)

# Loading the saved data into an edge table

In [ ]:
import glob

def load_collected_data(data_path, graph_object='edge'):
    data_df = pd.DataFrame()
    if graph_object == 'node':
        filestring = '_userinfo'
    elif graph_object == 'edge':
        filestring = '_mentions'
    else:
        print('Type unknown. graph_type only accept "node" or "edge".')
        raise
            
    for filename in glob.glob(data_path + '*' + filestring + '*' + '.json'):
        new_data_df = pd.read_json(filename)
        #print('{} with {} tweets.'.format(filename,len(new_data_df)))
        data_df = data_df.append(new_data_df)
    data_df.reset_index(drop=True, inplace=True)
    return data_df
    

In [ ]:
edge_df = load_collected_data(tweet_data_path, graph_object='edge')
node_df = load_collected_data(tweet_data_path, graph_object='node')
node_df = node_df[['user','name','user_details','all_hashtags']]
node_df.drop_duplicates(subset='user', inplace=True)
node_df.set_index('user', inplace=True)

## Creating the graph

In [ ]:
import pysad.graph
import pysad.clusters

In [ ]:
import importlib
importlib.reload(pysad.graph)
importlib.reload(pysad.collect)

In [ ]:
DEGREE_MIN = 2 # Minimal number of connections in the graph

G = pysad.graph.graph_from_edgeslist(edge_df)
#G = pysad.graph.graph_from_edgeslist(df_pop,DEGREE_MIN)
G = pysad.graph.add_node_attributes(G,node_df)
G = pysad.graph.reduce_graph(G,DEGREE_MIN)
G.name = category_name
G.end_date = max(edge_df['date']) #max(edge_df['date'].apply(max))
G.start_date = min(edge_df['date']) #min(edge_df['date'].apply(min))
print('Period from {} to {}.'.format(G.start_date,G.end_date))

In [ ]:
# Complete the info of the nodes not collected
nodes_missing_info = [node for node,data in G.nodes(data=True) if 'name' not in data]

In [ ]:
nodeinfo_df = pd.DataFrame()
for node in nodes_missing_info:
    print('collecting info for',node)
    edgeinfo,nodeinfo = pysad.collect.collect_user_data(node,python_tweets,max_day_old)
    nodeinfo_df = nodeinfo_df.append(nodeinfo)
nodeinfo_df = nodeinfo_df[['user','name','user_details','all_hashtags']]
nodeinfo_df.drop_duplicates(subset='user', inplace=True)
nodeinfo_df.set_index('user', inplace=True)

In [ ]:
G = pysad.graph.add_node_attributes(G,nodeinfo_df)

In [ ]:
# Check intergity
for node,data in G.nodes(data=True):
    if 'name' not in data:
        print('Missing information for',node)

## Community detection to get the clusters

In [ ]:
import importlib
importlib.reload(pysad.clusters)
importlib.reload(pysad.graph)

In [ ]:
G,clusters = pysad.graph.detect_communities(G)
G.nb_communities = len(clusters)
c_connectivity = pysad.clusters.cluster_connectivity(G)

In [ ]:
G = pysad.graph.remove_small_communities(G,clusters,min_size=5)

In [ ]:
# Save the graph
import networkx as nx

graphname = 'missinggraph_reddit'
graphfilename = results_data_path + graphname + '_t' + str(min_mentions) + '_md' + str(DEGREE_MIN) +'_graph.gexf'
nx.write_gexf(G,graphfilename)
print('Wrote',graphfilename)

In [ ]:
c_connectivity

## Analyze clusters

In [ ]:
import importlib
importlib.reload(pysad.clusters)

## Automatic processing of all clusters

In [ ]:
# Extracting the data from the clusters
cluster_info_dic = {}
for c_id in clusters:
    cgraph = clusters[c_id]
    cgraph = pysad.clusters.cluster_attributes(cgraph)
    table_dic = pysad.clusters.cluster_tables(cgraph)
    node_details = 
    cluster_filename = results_data_path + 'cluster' + str(c_id)
    cluster_info_dic[c_id] = {}
    cluster_info_dic[c_id]['info_table'] = table_dic
    cluster_info_dic[c_id]['user_details'] = node_details
    cluster_info_dic[c_id]['filename'] = cluster_filename    

In [ ]:
# Adding global infos
# keywords
corpus = pysad.clusters.get_corpus(cluster_info_dic)
keyword_dic = pysad.clusters.tfidf(corpus)

In [ ]:
# gathering global info
# Saving in excel files
for c_id in cluster_info_dic:
    info_table = cluster_info_dic[c_id]['info_table']
    info_table['keywords'] = keyword_dic[c_id]
    cluster_general_info = {'cluster id': c_id, 'Nb users': clusters[c_id].number_of_nodes(), 
                           'Nb of tweets':clusters[c_id].size(weight='weight'),
                           'Start date': str(G.start_date),
                           'End date': str(G.end_date),
                           'Search topic': category_name,
                           'cluster connectivity': c_connectivity[c_id]}
    cluster_general_df = pd.DataFrame.from_dict([cluster_general_info])
    #info_table = {'cluster':cluster_general_df, **info_table}
    sheet1 = pd.concat([cluster_general_df,info_table['hashtags'],info_table['keywords']],axis=1)
    tweet_table = info_table['text']
    #user_table = 
    cluster_indicators = pd.DataFrame([pysad.clusters.compute_cluster_indicators(clusters[c_id])])
    excel_data = {'cluster':sheet1, 'tweets':tweet_table, 'indicators': cluster_indicators, 'users': node_df}
    #excel_data = info_table
    pysad.clusters.save_excel(excel_data,cluster_info_dic[c_id]['filename'] + '_infos.xlsx', table_format='Fanny')
    pysad.graph.save_graph(clusters[c_id],cluster_info_dic[c_id]['filename'] + 'graph.gexf')

In [ ]:
info_table